In [1]:
import cobra
import difflib
from functools import reduce
import os
import warnings
cobra.Configuration.solver = 'glpk'
warnings.simplefilter("ignore", category=UserWarning)

def get_biocyc_id(met: cobra.Metabolite):
    biocyc_id = met.annotation.get('biocyc')
    if biocyc_id:
        biocyc_id = biocyc_id[biocyc_id.index(':')+1:]
    return biocyc_id

def replace_metabolite(reac: cobra.Reaction, old: cobra.Metabolite, new: cobra.Metabolite):
    coeff = reac.metabolites[old]
    reac.add_metabolites({old: -coeff, new: coeff})

def move_metabolite_compartments(model, source, target):
    obsolete_mets = []
    for m in model.metabolites:
        if m.id.endswith('_' + source):
            new = m.id[:-len(source)] + target
            print(m.id, '->', new)
            if new not in model.metabolites:
                m.id = new
                m.compartment = target
            else:
                for r in m.reactions:
                    replace_metabolite(r, m, new)
                obsolete_mets.append(m)
    model.remove_metabolites(obsolete_mets)

def metabolite_id_for_compartment(met: cobra.Metabolite, compartment: str) -> str:
    if met.id.endswith('_'+met.compartment): # this is the regular case
        return(met.id[:-len(m.compartment)] + compartment)
    else:
        idx = met.id.rfind('_')
        if idx < 0:
            raise ValueError
        else:
            return(met.id[:idx+1] + compartment)

In [2]:
species = 'lumbricus_terrestris'

In [3]:
icel = cobra.io.read_sbml_model("wormflux_iCEL1314/iCEL1314.xml.gz")

In [4]:
lru = cobra.io.read_sbml_model(os.path.join(species, "MetaFlux/LRU.sbml.gz"))

In [5]:
model = cobra.io.read_sbml_model("lumbricus_rubellus/MetaFlux/SBML_export.xml")
print(model.slim_optimize())
# prevent free ATP generation (check via ATPASE_RXN)
model.reactions.RXN_10862.lower_bound = 0
model.reactions.RXN_14218.lower_bound = 0
model.slim_optimize()

8.241250050548985


0.9653119288522743

Validate the BiGG mapping (for uniqe IDs only) between LRU and iCEL1314 so far.

In [333]:
from collections import defaultdict
def annotation_map(items: cobra.DictList, key: str) -> defaultdict:
    anno_map = defaultdict(list)
    for it in items:
        a = it.annotation.get(key)
        if a and isinstance(a, str):
            anno_map[a].append(it.id)
        # else:
        #     print(it.id, a)
    return anno_map

In [334]:
# update some annotations
icel.metabolites.get_by_id('g6p-A_c').annotation['bigg.metabolite'] = 'g6p_A'
icel.metabolites.get_by_id('g6p-B_c').annotation['bigg.metabolite'] = 'g6p_B'

In [335]:
icel_bigg_compounds = annotation_map(icel.metabolites, 'bigg.metabolite')

In [336]:
# add ETF (basically an equivalent of FAD) annotations
# in the LRU model FAD is only a metabolite in its synthesis pathway while ETF are the electron carriers
lru.metabolites.ETF_Reduced_c.annotation['bigg.metabolite'] = 'etfrd'
#lru.metabolites.get_by_id('ETF_Reduced_CCO-IN').annotation['bigg.metabolite'] = 'etfrd'
lru.metabolites.get_by_id('ETF_Reduced_c').annotation['bigg.metabolite'] = 'etfrd'
lru.metabolites.ETF_Oxidized_c.annotation['bigg.metabolite'] = 'etfox'
#lru.metabolites.get_by_id('ETF_Oxidized_CCO-IN').annotation['bigg.metabolite'] = 'etfox'
lru.metabolites.get_by_id('ETF_Oxidized_c').annotation['bigg.metabolite'] = 'etfox'

In [337]:
metabolite_compartments = defaultdict(set)
lru2icel = defaultdict(set) # maping form BioCyc ID to iCEL1314 metabolites
#icel2lru = dict() # mapping from LRU metabolite to iCEL1314 metabolite

In [338]:
for m in lru.metabolites:
    biocyc_id = m.annotation.get('biocyc')
    if biocyc_id:
        biocyc_id = biocyc_id[biocyc_id.index(':')+1:]
    else:
        continue
    bigg_id = m.annotation.get('bigg.metabolite', None)
    if bigg_id and isinstance(bigg_id, str):
        bigg_id = bigg_id.replace('__', '-') # BioCyc uses e.g. glc__D, iCEL1314 glc-D
        for icel_met in icel.metabolites.get_by_any(icel_bigg_compounds.get(bigg_id, [])):
            if m.charge != icel_met.charge:
                d = m.charge - icel_met.charge
                el = defaultdict(int)
                el.update(icel_met.elements)
                el['H'] += d
                if el == m.elements: # protonation difference only
                    print(m.id, bigg_id, icel_met.id, d)
                else:
                    print(m.id, bigg_id, icel_met.id)
            elif m.elements != icel_met.elements:
                print(m.id, bigg_id, icel_met.id, m.formula, icel_met.formula)
            metabolite_compartments[biocyc_id].add(icel_met.compartment)
            lru2icel[biocyc_id].add(icel_met)

3_HYDROXY_ANTHRANILATE_c 3hanthrn 3hanthrn_c -1
3_HYDROXY_ANTHRANILATE_c 3hanthrn 3hanthrn_e -1
3_SULFINOALANINE_c 3sala 3sala_m 1
3_SULFINOALANINE_c 3sala 3sala_c 1
5_PHOSPHORIBOSYL_5_AMINOIMIDAZOLE_c air air_c 1
ALA_tRNAs_c trnaala trnaala_c
ALA_tRNAs_c trnaala trnaala_m
ASP_tRNAs_c trnaasp trnaasp_c
ASP_tRNAs_c trnaasp trnaasp_m
CO3_c hco3 hco3_e
CO3_c hco3 hco3_c
CO3_c hco3 hco3_m
CPD_13612_c sphgn sphgn_ce_c C18H40NO2 C19H42NO2
CPD_13612_c sphgn sphgn_ce_e C18H40NO2 C19H42NO2
CPD_16459_c pi pi_e 1
CPD_16459_c pi pi_c 1
CPD_16459_c pi pi_m 1
CPD_649_c sph1p sph1p_ce_c C18H39NO5P C19H41NO5P
CPD_649_c sph1p sph1p_ce_e C18H39NO5P C19H41NO5P
CPD_678_c seln seln_c 1
CPD_846_c h2s h2s_m -1
CPD_846_c h2s h2s_c -1
DEHYDROSPHINGANINE_c 3dsphgn 3dsphgn_ce_c C18H38NO2 C19H40NO2
ETF_Oxidized_c etfox etfox_m
ETF_Reduced_c etfrd etfrd_m C13H13N4O2Wm RH2
FAD_c fad fad_e -1
FAD_c fad fad_c -1
FAD_c fad fad_m -1
FMN_c fmn fmn_e -1
FMN_c fmn fmn_c -1
FMN_c fmn fmn_m -1
H2CO3_c hco3 hco3_e 1
H2CO3_c 

In [339]:
lru2icel['Glucopyranose']

{<Metabolite glc-D-A_c at 0x7fb861ed1b10>,
 <Metabolite glc-D-B_c at 0x7fb86122d590>,
 <Metabolite glc-D_c at 0x7fb861a70a50>,
 <Metabolite glc-D_e at 0x7fb86121bad0>}

In [340]:
lru2icel['Glucopyranose'] = set(icel.metabolites.get_by_any(['glc-D_e', 'glc-D_e'])) # do not map to the alpha/beta instances

Next step: use KEGG mappings to find additional relations between LRU and iCEL1314

In [341]:
icel_kegg_compounds = annotation_map(icel.metabolites, 'kegg.compound')

In [342]:
for m in lru.metabolites:
    biocyc_id = m.annotation.get('biocyc')
    if biocyc_id:
        biocyc_id = biocyc_id[biocyc_id.index(':')+1:]
    else:
        continue
    kegg_id = m.annotation.get('kegg.compound', None)
    if kegg_id and isinstance(kegg_id, str):
        for icel_met in icel.metabolites.get_by_any(icel_kegg_compounds.get(kegg_id, [])):
            if m.charge != icel_met.charge:
                d = m.charge - icel_met.charge
                el = defaultdict(int)
                el.update(icel_met.elements)
                el['H'] += d
                if el == m.elements: # protonation difference only
                    print(m.id, kegg_id, icel_met.id, d)
                else:
                    print(m.id, kegg_id, icel_met.id)
            elif m.elements != icel_met.elements:
                print(m.annotation.get('biocyc', m.id), kegg_id, icel_met.id, m.formula, icel_met.formula)
            lru2icel[biocyc_id].add(icel_met)
            metabolite_compartments[biocyc_id].add(icel_met.compartment)
            lru2icel[biocyc_id].add(icel_met)

LRT:1-PHOSPHATIDYL-1D-MYO-INOSITOL-34-BISPH C11554 pail34p_c C11H14O19RRP3 C46.1308H77.1556O19.0P3.0
LRT:1-PHOSPHATIDYL-1D-MYO-INOSITOL-35-BISPH C11556 pail35p_c C11H14O19RRP3 C46.1308H77.1556O19.0P3.0
LRT:1-PHOSPHATIDYL-1D-MYO-INOSITOL-5-PHOSPHA C11557 pail5p_c C11H15O16RRP2 C46.1308H78.1556O16.0P2.0
3_HYDROXY_ANTHRANILATE_c C00632 3hanthrn_c -1
3_HYDROXY_ANTHRANILATE_c C00632 3hanthrn_e -1
3_SULFINOALANINE_c C00606 3sala_m 1
3_SULFINOALANINE_c C00606 3sala_c 1
5_PHOSPHORIBOSYL_5_AMINOIMIDAZOLE_c C03373 air_c 1
LRT:ACYL-COA C00040 Rtotalcoa_c C22H31N7O17RP3S C39.5654H62.5778N7.0O17.0P3.0S1.0
ALA_tRNAs_c C01635 trnaala_c
ALA_tRNAs_c C01635 trnaala_m
AMMONIA_c C00014 nh4_e -1
AMMONIA_c C00014 nh4_c -1
AMMONIA_c C00014 nh4_m -1
ASP_tRNAs_c C01638 trnaasp_c
ASP_tRNAs_c C01638 trnaasp_m
LRT:CARDIOLIPIN C05980 clpn_c C13H16O17RRRRP2 C83.2616H142.3112O17.0P2.0
LRT:CDPDIACYLGLYCEROL C00269 cdpdag_c C14H17N3O15RRP2 C49.1308H80.1556N3.0O15.0P2.0
LRT:CDPDIACYLGLYCEROL C00269 cdpdag_m C14H17N3O15

In [343]:
len(lru2icel)

449

In [344]:
print(lru2icel['Glucopyranose'])
print(lru2icel['ALPHA-GLUCOSE'])
print(lru2icel['GLC'])
lru2icel['GLC-6-P'] # should be beta-glucose

{<Metabolite glc-D_e at 0x7fb86121bad0>, <Metabolite glc-D_c at 0x7fb861a70a50>}
{<Metabolite glc-D-A_c at 0x7fb861ed1b10>}
{<Metabolite glc-D-B_c at 0x7fb86122d590>}


{<Metabolite g6p-B_c at 0x7fb86138cc10>}

In [345]:
import re
for k,v in lru2icel.items():
    lru_met = lru.metabolites.query(re.compile("^"+k.replace('-', '_').replace('+', '\+')+"_.*"), "id")[0]
    lru_name = lru_met.name
    icel_name = {m.name for m in v}
    if len(icel_name) != 1:
        print(k, lru_name, v)
        continue
    icel_name = icel_name.pop()
    synonyms = lru_met.annotation.get('metacyc.synonyms')
    if not synonyms:
        synonyms = []
    elif not isinstance(synonyms, list):
        synonyms = [synonyms]
    best_lru_match = difflib.get_close_matches(icel_name, [lru_name] + synonyms, cutoff=0, n=1)[0]
    similarity = difflib.SequenceMatcher(a=best_lru_match, b=icel_name).ratio()
    if similarity < 0.8:
        print(k, best_lru_match, " | ", icel_name, round(similarity, 2), [lru_name] + synonyms)

2-AMINO-MUCONATE (2Z,4E)-2-aminomuconate  |  2-Aminomuconate 0.74 ['(2Z,4E)-2-aminomuconate', '2-aminomuconic acid', '(2Z,4E)-2-aminohexa-2,4-dienedioate']
ADENYLOSUCC <i>N<sup>6</sup></i>-(1,2-dicarboxyethyl)AMP  |  "N6-(1,2-Dicarboxyethyl)-AMP" 0.68 ['adenylosuccinate', '<i>N<sup>6</sup></i>-(1,2-dicarboxyethyl)AMP', 'adenylosuccinic acid']
ALA-tRNAs a tRNAAla  |  tRNA(Ala) 0.78 ['a tRNAAla', 'TRNA(ALA)']
ALPHA-D-GALACTOSE &alpha;-D-galactose  |  D-Galactose 0.67 ['alpha-D-galactopyranose', '6-(hydroxymethyl)tetrahydropyran-2,3,4,5-tetraol', '&alpha;-D-galactose', '&alpha;-D-Gal']
ASP-tRNAs tRNAasp  |  tRNA(Asp) 0.75 ['tRNAasp', 'TRNA(ASP)']
BETAINE_ALDEHYDE betaine aldehyde hydrate  |  Betaine aldehyde 0.75 ['betaine aldehyde hydrate', 'choline gem-diol', 'glycine betaine aldehyde hydrate']
CPD-10267 decanoyl-CoA  |  Decanoyl-CoA (n-C10:0CoA) 0.59 ['decanoyl-CoA']
CPD-15159 (6R)-L-erythro-6,7-dihydrobiopterin  |  "6,7-Dihydrobiopterin" 0.67 ['(6R)-L-erythro-6,7-dihydrobiopterin', '<

lru2icel mappings at this stage appear to be correct

In [346]:
def canonical_formula_charge(met: cobra.Metabolite) -> frozenset:
    canonical = {('charge', met.charge)}
    met_elements = met.elements
    if met_elements is None: # can also happen with formulas like C20H33N2O3SR(C2H2NOR)n
        canonical.add(met.formula)
    else:
        canonical.update((e, c) for e,c in met_elements.items())
    return frozenset(canonical)

In [86]:
# icel_mets = defaultdict(set) # dictionary from formula and charge to iCEL1314 metabolites
# for m in icel.metabolites:
#     if not m.id.endswith('_' + m.compartment):
#         raise
#     icel_mets[canonical_formula_charge(m)].add(m.id[:-2])

In [87]:
# for m in lru.metabolites:
#     biocyc_id = get_biocyc_id(m)
#     if biocyc_id is None or biocyc_id in lru2icel:
#         continue
#     candidates = icel_mets[canonical_formula_charge(m)]
#     if len(candidates) > 0:
#         # if len(candidates) == 1:
#         #     candidates = candidates.pop()
#         #     print(biocyc_id, candidates)
#         # else:
#             synonyms = [m.name] + m.annotation['metacyc.synonyms']
#             # try:
#             #     pf = PFrame(biocyc_id, meta, getFrameData=True)
#             #     synonyms.extend(pf.synonyms)
#             # except:
#             #     try:
#             #         synonyms.extend(sendQueryToPTools(
#             #             f'(get-slot-values (nth 0 (with-organism (:org-id \'meta) (get-frame-labeled "{biocyc_id}"))) \'synonyms)'))
#             #     except:
#             #         pass
#             for c in candidates:
#                 icel_id = c + '_c'
#                 if icel_id not in icel.metabolites:
#                     icel_id = c + '_m' # try only with metabolites that (also) occur in mitochondrion?
#                 mi = icel.metabolites.get_by_id(icel_id)
#                 close_matches = difflib.get_close_matches(mi.name, synonyms, cutoff=0.75, n=1)
#                 if len(close_matches) > 0:
#                     print(biocyc_id, mi.name, close_matches, mi.id, synonyms) # check manually if mi.name could be a close match

In [347]:
icel_mets = defaultdict(set) # dictionary from formula and charge to iCEL1314 metabolites
for m in icel.metabolites:
    if not m.id.endswith('_' + m.compartment):
        raise
    icel_mets[canonical_formula_charge(m)].add(m)

In [348]:
# could also manually add BiGG mapping
lru2icel['Pi'] = set(icel.metabolites.get_by_any(['pi_c', 'pi_m', 'pi_e']))
metabolite_compartments['Pi'] = {'c', 'm', 'e'}

# # iCEL1314 distinguishes between FAD and ETF in the mitochondrion but they should be unified
# lru2icel['FAD'].add(icel.metabolites.get_by_id('etfox_m'))
# lru2icel['ETF-Oxidized'] = lru2icel['FAD'] # !! ETF_ has formula with residual group, needs to be updated later !!
# lru2icel['ETF-Reduced'] = set(icel.metabolites.get_by_any(['fadh2_m', 'etfrd_m']))
# metabolite_compartments['FAD'] = {'c', 'm'}
# metabolite_compartments['ETF-Oxidized'] = {'c', 'm'}
# metabolite_compartments['ETF-Reduced'] = {'m'}

mapped_icel_mets = reduce(set.union, lru2icel.values())

Show potential further metabolite mappings.

In [90]:
# tags = re.compile('<.*?>')
# for ml in lru.metabolites:
#     biocyc_id = get_biocyc_id(ml)
#     if biocyc_id is None or biocyc_id in lru2icel:
#         continue
#     candidates = set()
#     mi_names = set()
#     for mi in (icel_mets[canonical_formula_charge(ml)] - mapped_icel_mets):
#         try:
#             mi_name = mi.name.replace('"', '') # quotes can mess up the LISP expression
#             mi_names.add(mi_name)
#             frames = sendQueryToPTools(
#                 f'(get-slot-values (nth 0 (with-organism (:org-id \'meta) (get-frame-labeled "{mi_name}"))) \'synonyms)')
#             if frames:
#                 candidates.update({tags.sub('', f) for f in frames} - lru2icel.keys())
#         except PToolsError:
#             continue
#     if len(candidates):
#         print(biocyc_id, mi_names, candidates)

D-Ribopyranose might be mapped but is not important as it is only cytosolic

In [349]:
# metabolites only in mitochondrion according to iCEL1314
only_mitochondrial_metabolites = [k for k,v in lru2icel.items() if not any(m.compartment == 'c' for m in v) and any(m.compartment == 'm' for m in v)]
len(only_mitochondrial_metabolites)

34

In [350]:
only_mitochondrial_metabolites = set(only_mitochondrial_metabolites)
only_mitochondrial_metabolites.update(
    [m.id for m in lru.metabolites if m.compartment == 'm' and metabolite_id_for_compartment(m, 'c') not in lru.metabolites])

In [351]:
fully_mapped_reactions = [r for r in lru.reactions if all(get_biocyc_id(m) in lru2icel for m in r.metabolites)]
len(fully_mapped_reactions)

591

In [352]:
mitochondrial = [r for r in fully_mapped_reactions if not r.boundary and all('m' in metabolite_compartments[get_biocyc_id(m)] for m in r.metabolites)]
len(mitochondrial) # acutally possibly mitochondrial only, a corresponding mitochondrial reaction should also be in iCEL1314

192

In [353]:
lru2icel_mit = dict()
lru2icel_cyt = dict()
for ml, mi in lru2icel.items():
    mi_mit = [m for m in mi if m.compartment == 'm']
    if len(mi_mit):
        lru2icel_mit[ml] = mi_mit[0]
    mi_cyt = [m for m in mi if m.compartment == 'c']
    if len(mi_cyt):
        lru2icel_cyt[ml] = mi_cyt[0]
print(len(lru2icel_mit), len(lru2icel_cyt))

207 415


In [354]:
icel_cyto_and_mito_rxns = {r.id[2:] for r in icel.reactions if r.id.startswith('RM') and ('RC' + r.id[2:]) in icel.reactions}
for r in icel_cyto_and_mito_rxns:
    rc = icel.reactions.get_by_id('RC' + r)
    rm = icel.reactions.get_by_id('RM' + r)
    mc = {m.id[:-2] for m in rc.metabolites}
    mm = {m.id[:-2] for m in rm.metabolites}
    if mc != mm:
        print(r, mc - mm, mm - mc)

C0097 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0089 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0098 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0103 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0107 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0106 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0094 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0093 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0088 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0091 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0126 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0104 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0090 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0095 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0096 {'o2', 'h2o2'} {'fadh2', 'fad'}
C0092 {'o2', 'h2o2'} {'fadh2', 'fad'}


It appears that the differences affect only cytosolic (or rather peroximsomal) and mitochondrial beta-oxidation.

In [355]:
# these may or may not be exclusively mitochondrial
# it is only checked that a reaction in iCEL1314 exists that has the same metabolites,
# except for protons
mito_rxns = set() # in case there are isozymes in iCEL1314
only_mito_rxns = set()
lru_rxn2icel_rxn = defaultdict(list)
for r in lru.reactions:
    if not r.boundary and all(get_biocyc_id(m) in lru2icel_mit for m in r.metabolites):
        rxns = [{ri for ri in lru2icel_mit[get_biocyc_id(m)].reactions if ri.id.startswith("RM")} for m in r.metabolites if m.name != 'H+']
        if len(rxns):
            rxns = reduce(set.intersection, rxns)
            for ri in rxns:

                mi = list(ri.metabolites)
                for ml in r.metabolites:
                    if ml.name != 'H+' and lru2icel_mit[get_biocyc_id(ml)] in mi:
                        mi.remove(lru2icel_mit[get_biocyc_id(ml)])
                if len(mi) == 0 or (len(mi) == 1 and mi[0].name == 'H+'):
                    mito_rxns.add(r.id)
                    lru_rxn2icel_rxn[r].append(ri.id)
                    if ('RC' + ri.id[2:]) not in icel.reactions:
                        only_mito_rxns.add(r.id)
                        print("* ", end="")
                    print(r.id, r.build_reaction_string(), ri.id, ri.build_reaction_string())

                # if len(r.metabolites) == len(ri.metabolites) and all(lru2icel_mit[get_biocyc_id(m)] in ri.metabolites for m in r.metabolites):
                #     mito_rxns.add(r.id)
                #     if ('RC' + ri.id[2:]) not in icel.reactions:
                #         only_mito_rxns.add(r.id)
                #         print("* ", end="")
                #     print(r.id, r.build_reaction_string(), ri.id, ri.build_reaction_string())

print(len(mito_rxns), len(only_mito_rxns))                    

* PYRROLINECARBREDUCT_RXN_PRO/NADP//L_DELTA1_PYRROLINE_5_CARBOXYLATE/NADPH/PROTON.56. L_DELTA1_PYRROLINE_5_CARBOXYLATE_c + NADPH_c + 2.0 PROTON_c --> NADP_c + PRO_c RM01251 1pyr5c_m + 2.0 h_m + nadph_m --> nadp_m + pro-L_m
* ACYLCOADEHYDROG_RXN_PALMITYL_COA/ETF_Oxidized/PROTON//CPD0_2117/ETF_Reduced.56. ETF_Oxidized_c + PALMITYL_COA_c + PROTON_c --> CPD0_2117_c + ETF_Reduced_c RM01282 etfox_m + pmtcoa_m <=> etfrd_m + hdd2coa_m
* PANTEPADENYLYLTRAN_RXN ATP_c + PANTETHEINE_P_c + PROTON_c --> DEPHOSPHO_COA_c + PPI_c RM03035 atp_m + h_m + pan4p_m <=> dpcoa_m + ppi_m
* RXN_14116 L_GLUTAMATE_GAMMA_SEMIALDEHYDE_c + NAD_c + WATER_c --> GLT_c + NADH_c + 2.0 PROTON_c RM00245 glu5sa_m + h2o_m + nad_m --> glu-L_m + 2.0 h_m + nadh_m
* RXN0_2301 ETF_Oxidized_c + ISOVALERYL_COA_c + PROTON_c --> 3_METHYL_CROTONYL_COA_c + ETF_Reduced_c RM04096 etfox_m + ivcoa_m --> 3mb2coa_m + etfrd_m
4_HYDROXYGLUTAMATE_AMINOTRANSFERASE_RXN 2_KETOGLUTARATE_c + L_ERYTHRO_4_HYDROXY_GLUTAMATE_c <=> D_4_HYDROXY_2_KETO_GLUT

In [356]:
for r in lru.reactions:
    if not r.boundary and all(get_biocyc_id(m) in lru2icel_cyt for m in r.metabolites):
        rxns = [{ri for ri in lru2icel_cyt[get_biocyc_id(m)].reactions if ri.id.startswith("RC")} for m in r.metabolites if m.name != 'H+']
        if len(rxns):
            rxns = reduce(set.intersection, rxns)
            for ri in rxns:

                mi = list(ri.metabolites)
                for ml in r.metabolites:
                    if ml.name != 'H+' and lru2icel_cyt[get_biocyc_id(ml)] in mi:
                        mi.remove(lru2icel_cyt[get_biocyc_id(ml)])
                if len(mi) == 0 or (len(mi) == 1 and mi[0].name == 'H+'):
                    lru_rxn2icel_rxn[r].append(ri.id)

ATPSYN_RXN is not contained in mito_rxns

In [357]:
cyt_mit_trans = []
for r in icel.reactions:
    if r.id.startswith("TCM"):
        if all(m in mapped_icel_mets for m in r.metabolites):
            cyt_mit_trans.append(r)
            #print(r.build_reaction_string())
len(cyt_mit_trans)

183

In [358]:
lru = cobra.io.read_sbml_model(os.path.join(species, "MetaFlux/LRU.sbml.gz"))

In [359]:
# add ETF (basically an equivalent of FAD) annotations
# in the LRU model FAD is only a metabolite in its synthesis pathway while ETF are the electron carriers
lru.metabolites.ETF_Reduced_c.annotation['bigg.metabolite'] = 'etfrd'
#lru.metabolites.get_by_id('ETF_Reduced_CCO-IN').annotation['bigg.metabolite'] = 'etfrd'
lru.metabolites.ETF_Oxidized_c.annotation['bigg.metabolite'] = 'etfox'
#lru.metabolites.get_by_id('ETF_Oxidized_CCO-IN').annotation['bigg.metabolite'] = 'etfox'

In [208]:
# # add BiGG annotations for ETF_Reduced
# lru.metabolites.get_by_id('ETF_Reduced_c').annotation['bigg.metabolite'] = 'fadh2'
# lru.metabolites.get_by_id('ETF_Reduced_CCO-IN').annotation['bigg.metabolite'] = 'fadh2'

In [209]:
# # unify ETF-Oxidized and FAD
# for r in lru.metabolites.get_by_id('ETF_Oxidized_c').reactions:
#     replace_metabolite(r, lru.metabolites.get_by_id('ETF_Oxidized_c'), lru.metabolites.get_by_id('FAD_c'))
# lru.remove_metabolites([lru.metabolites.get_by_id('ETF_Oxidized_c')])
# lru.metabolites.get_by_id('ETF_Oxidized_CCO-IN').id = 'FAD_CCO-IN'
# lru.metabolites.get_by_id('FAD_CCO-IN').name = lru.metabolites.FAD_c.name
# # TODO: also copy formula, charge
# lru.metabolites.get_by_id('FAD_CCO-IN').annotation = lru.metabolites.FAD_c.annotation

In [360]:
lru.compartments

{'c': 'CCO-CYTOSOL',
 'm': 'CCO-MIT-LUM',
 'T': 'T',
 'CCO__45__MIT__45__IMEM': 'CCO-MIT-IMEM',
 'CCO__45__IN': 'CCO-IN',
 'CCO__45__OUT': 'CCO-OUT',
 'e': 'CCO-EXTRACELLULAR'}

In [361]:
additional_mito_rxns = [r for r in lru.reactions.list_attr('id') if 'CCO_MIT_LUM' in r]

In [362]:
# TODO: are these only mitochondrial?
additional_mito_rxns += [
    '2OXOGLUTDECARB_RXN', 'RXN0_1147', 'RXN_7716', # 2-oxoglutarate dehydrogenase is split into three subreactions
    'SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_SUC/UBIQUINONE_8//FUM/CPD_9956.31.', # missing because of incomplete quinone mapping
    'RXN_20154', 'RXN_20155' # subreactions of acetyl-CoA-acetyltransferase
]
# move beta-oxidation (reactions of pathway FAO-PWY) of regular FA
# into mitochondrion (peroxisome, if present, is probably used for (very-) long chain FA)
for r in lru.reactions:
    if 'FAO-PWY' in r.annotation.get('biocyc.pathway', []):
        additional_mito_rxns.append(r.id)
        only_mito_rxns.add(r.id)

iCEL1314 has lumped palmitate synthesis in cytsol (RCC0020) and beta-oxidation also in the cytosol. There are some FA-related reactions in the mitochondrion. ACP metabolites are not in this model.

In [366]:
# make decarboxylases irreversible (most already are)
if species == 'lumbricus_rubellus':
    for r in lru.reactions.get_by_any([
        'ASPDECARBOX_RXN', 'PHOSPHASERDECARB_RXN', 'SULFINOALANINE_DECARBOXYLASE_RXN']):
        r.lower_bound = 0

In [367]:
# make some reactions from respiration irreversible in their physiological direction
for r in lru.reactions.get_by_any([
    'RXN_15829_CPD_9956/Oxidized_CycA1_cytochromes/PROTON//UBIQUINONE_8/Reduced_CycA1_cytochromes/PROTON.90.', 'NADH_DEHYDROG_A_RXN_NADH/UBIQUINONE_8/PROTON//NAD/CPD_9956/PROTON.46.']):
    r.lower_bound = 0

In [368]:
# properly name exchange reactions (with BiGG ID where possible)
for r in lru.boundary:
    m = list(r.metabolites.keys())[0]
    r.id = 'EX_' + m.annotation.get('bigg.metabolite', m.id) + '_' + m.compartment

Pyruvate is transported into the mitochondrion and PDC produces Acetyl-CoA there. A malate/aspartate shuttle transports NADH from the cytosol to the mitochondrion lumen.

In iCEL1314 respiration and ATP synthase exchange protons between mitochondrion and cytosol, not the intermembrane space. 46 ATP per glucose are being produced which is too high.

In [369]:
for m in lru.metabolites:
    biocyc_id = get_biocyc_id(m)
    # if biocyc_id in only_mitochondrial_metabolites:
    #     m.id = metabolite_id_for_compartment(m, 'm')
    #     m.compartment = 'm'
    # elif biocyc_id in lru2icel and any(mi.compartment == 'm' for mi in lru2icel[biocyc_id]):
    if biocyc_id in lru2icel and any(mi.compartment == 'm' for mi in lru2icel[biocyc_id]): # always copy to prevent reactions with mixed metabolites
        mm = m.copy()
        mm.id = metabolite_id_for_compartment(mm, 'm')
        mm.compartment = 'm'
        lru.add_metabolites([mm])

In [370]:
[m.id for m in lru.metabolites if m.compartment == 'T']

['CPD_9956_T', 'UBIQUINONE_8_T']

In [371]:
[m.id for m in lru.metabolites if m.compartment == 'CCO__45__OUT']

['Oxidized_CycA1_cytochromes_CCO-OUT',
 'PROTON_CCO-OUT',
 'Reduced_CycA1_cytochromes_CCO-OUT']

In [372]:
[m.id for m in lru.metabolites if m.compartment == 'CCO__45__MIT__45__IMEM']

['CPD_9956_CCO-MIT-IMEM', 'UBIQUINONE_8_CCO-MIT-IMEM']

compartment CCO-OUT-CCO-IN does not contain any metabolites

In [373]:
move_metabolite_compartments(lru, "CCO-IN", "m")
move_metabolite_compartments(lru, "T", "m")

FUM_CCO-IN -> FUM_m
PROTON_CCO-IN -> PROTON_m
SUC_CCO-IN -> SUC_m
CPD_9956_T -> CPD_9956_m
UBIQUINONE_8_T -> UBIQUINONE_8_m


rename both CCO-OUT and CCO-MIT-IMEM to CCO_MIT_IMEM, otherwise there will be an issue with loading/saving of the model

In [374]:
move_metabolite_compartments(lru, "CCO-OUT", "CCO_MIT_IMEM")

Oxidized_CycA1_cytochromes_CCO-OUT -> Oxidized_CycA1_cytochromes_CCO_MIT_IMEM
PROTON_CCO-OUT -> PROTON_CCO_MIT_IMEM
Reduced_CycA1_cytochromes_CCO-OUT -> Reduced_CycA1_cytochromes_CCO_MIT_IMEM


In [375]:
move_metabolite_compartments(lru, "CCO-MIT-IMEM", "CCO_MIT_IMEM")

CPD_9956_CCO-MIT-IMEM -> CPD_9956_CCO_MIT_IMEM
UBIQUINONE_8_CCO-MIT-IMEM -> UBIQUINONE_8_CCO_MIT_IMEM


In [376]:
# add additional mitochondrial metabolites where necessary
for m in reduce(set.union, (set(r.metabolites) for r in lru.reactions.get_by_any(additional_mito_rxns))):
    mm_id = metabolite_id_for_compartment(m, 'm')
    if mm_id not in lru.metabolites:
        mm = m.copy()
        mm.id = mm_id
        mm.compartment = 'm'
        lru.add_metabolites([mm])

In [377]:
icel2lru = dict() # mapping of metabolite ID stems
for m in lru.metabolites:
    biocyc_id = get_biocyc_id(m)
    if biocyc_id in lru2icel:
        stem = {mi.id[:-len(mi.compartment)-1] for mi in lru2icel[biocyc_id]}
        if len(stem) != 1:
            print(biocyc_id, stem)
        for s in stem:
            icel2lru[s] = m.id[:-len(m.compartment)-1]
# icel2lru['fadh2'] = 'ETF_Reduced'

In [378]:
for r in cyt_mit_trans:
    rl = cobra.Reaction(r.id, name=r.name)
    rl.bounds = r.bounds
    lru.add_reactions([rl])
    rl.add_metabolites({icel2lru[m.id[:-len(m.compartment)-1]]+'_'+m.compartment:c for m,c in r.metabolites.items()})
    mb = rl.check_mass_balance()
    if len(mb):
        print(rl.id, mb, rl.build_reaction_string())

TCM0108 {'charge': 1.0, 'H': 1.0} ATP_c + PHOSPHATIDYLCHOLINE_m + WATER_c --> ADP_c + CPD_16459_c + PHOSPHATIDYLCHOLINE_c + PROTON_c
TCM2002 {'charge': 1.0, 'H': 1.0} ATP_c + L_1_PHOSPHATIDYL_SERINE_c + WATER_c --> ADP_c + CPD_16459_c + L_1_PHOSPHATIDYL_SERINE_m + PROTON_c


above can be fixed by removing the proton on RHS

In [379]:
for r in lru.reactions.get_by_any(list(mito_rxns.union(additional_mito_rxns))):
    if any(m.id.endswith("_CCO-OUT") for m in r.metabolites):
        print(r.id, r.build_reaction_string())
        continue
    rm = r.copy() # for now make copies in all cases
    rm.id = rm.id + '_m'
    lru.add_reactions([rm])
    for m in list(rm.metabolites.keys()): # make a list copy because rm.metabolites changes in this loop
        replace_metabolite(rm, m, lru.metabolites.get_by_id(metabolite_id_for_compartment(m, 'm')))
    

In [380]:
lru.reactions.EX_co2_c.bounds = (-1, 1000)

In [297]:
import os
os.environ['JAVA_HOME'] = 'E:\\mpi\\cnapy-1.2.3\\cnapy-environment\\Library'

In [381]:
from cnapy.core import replace_ids
replace_ids(lru.metabolites, 'bigg.metabolite', unique_only=True)

ACP_m remains unchanged
ACP_c remains unchanged


In [299]:
# # rename metabolites in reaction IDs of generic reactions
# for r in lru.reactions:
#     if r'/' in r.id:
#         new_id = r.id
#         for m in r.metabolites:
#             biocyc_id = get_biocyc_id(m)
#             if biocyc_id:
#                 new_id = new_id.replace(biocyc_id.replace('-', '_'), m.id[:-2])
#         r.id = new_id

ATP hydrolysis in the cytosol produces one proton while ATP synthesis in the mitochondrion takes up one proton. To ensure this is equibrilated the phospate transport between these compartments is modified (cf. Edmund RS Kunji, Alan J Robinson, Coupling of proton and substrate translocation in the transport cycle of mitochondrial carriers, Current Opinion in Structural Biology, 2010).

In BiGG the typical eukaryotic mitochondrial phosphate transport is also a proton symport.

In [382]:
lru.reactions.TCM1292.id = 'PIt2m'
lru.reactions.PIt2m.name = "Phosphate transporter mitochondrial"
lru.reactions.PIt2m.build_reaction_from_string("Pi_c + h_c <=> Pi_m + h_m")
lru.reactions.PIt2m.annotation.clear()

In [383]:
lru.remove_reactions(only_mito_rxns - {'ATPSYN_RXN'}, remove_orphans=True) # no copy of ATPSYN_RXN was made because of CCO-OUT metabolite

In [384]:
# there is only mitochondrial NAD-dependent pyruvate dehydrogenase (RM00209) in the iCEL1314 model
lru.reactions.get_by_id('PYRUVATE_DEHYDROGENASE_NADP+_RXN').id = 'PDHm'
lru.reactions.PDHm.name = 'Pyruvate dehydrogenase'
lru.reactions.PDHm.build_reaction_from_string("coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m")
lru.reactions.PDHm.annotation.clear()
lru.reactions.PDHm.annotation['kegg.reaction'] = 'R00209'

In [385]:
# In BiGG the typical eukaryotic pyruvate mitochondrial transport is via proton symport.
# Without the proton symport the model uses weird ways to equilibrate the proton imbalance.
lru.reactions.TCM1280.id = 'PYRt2m' 
lru.reactions.PYRt2m.name = "Pyruvate mitochondrial transport via proton symport"
lru.reactions.PYRt2m.build_reaction_from_string("pyr_c + h_c <=> pyr_m + h_m")
lru.reactions.PYRt2m.annotation.clear()

In [386]:
lru._compartments['CCO_MIT_IMEM'] = 'CCO-MIT-IMEM'

In [388]:
cobra.io.write_sbml_model(lru, os.path.join(species, "metabolic_model", "compartment_model.sbml.gz"))

In [389]:
lru = cobra.io.read_sbml_model(os.path.join(species, "metabolic_model", "compartment_model.sbml.gz"))

In [390]:
lru.slim_optimize()

5.386608297091999

In [391]:
# reactions without metabolites
for r in lru.reactions:
    if len(r.metabolites) == 0:
        print(r)

RXN_14812_FRUCTOSE_6P//FRUCTOSE_6P.25.:  <=> 


In [392]:
lru.remove_reactions(['RXN_14812_FRUCTOSE_6P//FRUCTOSE_6P.25.'])

In [393]:
for r in lru.reactions:
    if not r.boundary and all('bigg.metabolite' in m.annotation for m in r.metabolites) and all(m.id in icel.metabolites for m in r.metabolites if m.name != 'H+'):
        rxns = reduce(set.intersection, (set(icel.metabolites.get_by_id(m.id).reactions) for m in r.metabolites if m.name != 'H+'))
        lru_mets = {m.id for m in r.metabolites if m.name != 'H+'}
        candidates = []
        for ri in rxns:
            if lru_mets == {m.id for m in ri.metabolites if m.name != 'H+'}:
                candidates.append(ri.id)
        if len(candidates) != 1:
            print(r, candidates)
        r.annotation['iCEL1314.reaction'] = candidates
        if r.id == 'BUTYRYL_COA_DEHYDROGENASE_RXN_m':
            break

TRANS_2_ENOYL_COA_REDUCTASE_NAD+_RXN_STEAROYL_COA/NAD//CPD_10262/NADH/PROTON.40.: h_c + nadh_c + od2coa_c --> nad_c + stcoa_c []
RXN_9673: atp_c + coa_c + lnlc_c --> amp_c + lnlccoa_c + ppi_c []
TRANS_2_ENOYL_COA_REDUCTASE_NAD+_RXN_LAUROYLCOA_CPD/NAD//CPD_7222/NADH/PROTON.41.: dd2coa_c + h_c + nadh_c --> ddcacoa_c + nad_c []
RXN_10811: coa_c + h2o_c --> 2.0 h_c + pan4p_c + pap_c []
TRANS_2_ENOYL_COA_REDUCTASE_NAD+_RXN_PALMITYL_COA/NAD//CPD0_2117/NADH/PROTON.40.: h_c + hdd2coa_c + nadh_c --> nad_c + pmtcoa_c []
NAD_KIN_RXN: atp_c + nad_c --> adp_c + h_c + nadp_c []
ACETALD_DEHYDROG_RXN: acald_c + coa_c + nad_c --> accoa_c + h_c + nadh_c []
2.7.4.10_RXN_ITP/AMP//IDP/ADP.17.: amp_c + itp_c <=> adp_c + idp_c []
RXN_10745_NADH/PROTON/OXYGEN_MOLECULE//HYDROGEN_PEROXIDE/NAD.51.: h_c + nadh_c + o2_c --> h2o2_c + nad_c []
RIBOSYLNICOTINAMIDE_KINASE_RXN: atp_c + rnam_c --> adp_c + h_c + nmn_c []
RXN_7904_STEARIC_ACID/CO_A/ATP//STEAROYL_COA/AMP/PPI.44.: atp_c + coa_c + ocdca_c --> amp_c + ppi_c +

In [394]:
cobra.io.write_sbml_model(lru, os.path.join(species, "metabolic_model", "compartment_model.sbml.gz"))

ATPASE_RXN is present.

Some metabolites like ubiquinone are more specific in LRU compared to iCEL1314 and therefore not mapped, for these an 'iCEL1314.metabolite' annotation could be introduced.

PDH is not a complex in LRU because BioCyc models the three steps of the PDH complex separately as distinct reactions.

In [396]:
for r in lru.reactions:
    if not r.boundary:
        mb = r.check_mass_balance()
        if len(mb):
            print(r.id, mb, r.build_reaction_string())

BIOMASS_EQUATION {'charge': 0.8254660530000137, 'C': -6.564236705500065, 'H': -10.828701109000008, 'N': -2.076681466000011, 'O': -4.91405648600022, 'P': -0.6358068730000141, 'S': -0.078938609} 1e-05 CPD1G_277_c + 1e-05 CPD_22020_c + 0.070647635 ala__L_c + 0.011312218 amet_c + 0.040727146 arg__L_c + 0.033155132 asp__L_c + 20.03057 atp_c + 0.011312218 coa_c + 0.026018102 ctp_c + 0.012571015 cys__L_c + 0.010855213 datp_c + 0.01120361 dctp_c + 0.01120361 dgtp_c + 0.010855213 dttp_c + 0.011312218 fad_c + 0.036169235 gln__L_c + 0.036169235 glu__L_c + 0.084174335 gly_c + 0.03500503 gtp_c + 20.0 h2o_c + 1e-05 hdca_c + 0.013085618 his__L_c + 0.039918486 ile__L_c + 0.061972897 leu__L_c + 0.047122926 lys__L_c + 0.021098722 met__L_c + 0.011312218 nad_c + 0.011312218 nadp_c + 1e-05 ocdca_c + 0.025509601 phe__L_c + 0.030435089 pro__L_c + 0.011312218 pydx5p_c + 0.011312218 ribflv_c + 0.029626427 ser__L_c + 0.011312218 so4_c + 0.011312218 thmpp_c + 0.034845974 thr__L_c + 0.0077925585 trp__L_c + 0.0189

In [397]:
[m for m in lru.metabolites if len(m.reactions) == 0]

[<Metabolite trnaala_m at 0x7fb7cb31e090>,
 <Metabolite L2aadp6sa_m at 0x7fb7cb340f50>,
 <Metabolite nh3_m at 0x7fb7cb31ef90>,
 <Metabolite ARACHIDONYL_COA_m at 0x7fb7cb348c10>,
 <Metabolite trnaasp_m at 0x7fb7cb34bd50>,
 <Metabolite CO3_m at 0x7fb8583a1dd0>,
 <Metabolite 2mop_m at 0x7fb85839b990>,
 <Metabolite CPD_14018_m at 0x7fb85839bd10>,
 <Metabolite CPD_14394_m at 0x7fb858397190>,
 <Metabolite CPD_14407_m at 0x7fb858397510>,
 <Metabolite lnlccoa_m at 0x7fb858397c10>,
 <Metabolite CPD_1843_m at 0x7fb858382050>,
 <Metabolite CPD_846_m at 0x7fb858381950>,
 <Metabolite H2CO3_m at 0x7fb8583b2cd0>,
 <Metabolite HSO3_m at 0x7fb8583b6c10>,
 <Metabolite OLEOYL_COA_m at 0x7fb85838bf90>,
 <Metabolite pppi_m at 0x7fb7cbf8d5d0>,
 <Metabolite Red_Thioredoxin_m at 0x7fb7cbf86050>,
 <Metabolite trnatrp_m at 0x7fb7cbf92650>,
 <Metabolite trnatyr_m at 0x7fb7cbf837d0>]

In [3]:
lru = cobra.io.read_sbml_model(os.path.join(species, "metabolic_model", "compartment_model.sbml.gz"))

correct missing mitochondrial intermembrane metabolites

In [4]:
m = lru.metabolites.h_c.copy()
m.id = 'h_i'
m.compartment = 'i'
lru.add_metabolites([m])
lru._compartments['i'] = 'mitochondrial intermembrane space'

In [5]:
lru.reactions.get_by_id('ATPSYN_RXN[CCO_MIT_IMEM]_ATP/WATER/PROTON//ADP/Pi/PROTON.46.').add_metabolites(
    {lru.metabolites.h_m: 4, lru.metabolites.h_i: -4})

In [6]:
for r in lru.reactions:
    if r.id.startswith('1.10.2.2_RXN'):
        r.add_metabolites({lru.metabolites.h_m: -4, lru.metabolites.h_i: 4})
        print(r.build_reaction_string())

2.0 Cytochromes_C_Oxidized_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Cytochromes_C_Reduced_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_CycA1_cytochromes_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_CycA1_cytochromes_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_cytochromes_C4_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_cytochromes_C4_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 an_oxidized_NrfB_protein_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 a_reduced_NrfB_protein_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_NapC_proteins_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_NapC_proteins_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_CymA_Proteins_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_CymA_Proteins_m + 4 h_i + q8_CCO_MIT_IMEM


In [7]:
for r in lru.reactions:
    if r.id.startswith('RXN_15816'):
        r.add_metabolites({lru.metabolites.h_m: -4, lru.metabolites.h_i: 4})
        print(r.build_reaction_string())

2.0 an_oxidized_NrfB_protein_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 a_reduced_NrfB_protein_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_CymA_Proteins_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_CymA_Proteins_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_cytochromes_C4_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_cytochromes_C4_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_CycA1_cytochromes_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_CycA1_cytochromes_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Oxidized_NapC_proteins_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Reduced_NapC_proteins_m + 4 h_i + q8_CCO_MIT_IMEM
2.0 Cytochromes_C_Oxidized_m + 2.0 h_m + q8h2_CCO_MIT_IMEM --> 2.0 Cytochromes_C_Reduced_m + 4 h_i + q8_CCO_MIT_IMEM


In [8]:
lru.reactions.get_by_id('NADH_DEHYDROG_A_RXN_NADH/UBIQUINONE_8/PROTON//NAD/CPD_9956/PROTON.46.').add_metabolites(
    {lru.metabolites.h_m: -4, lru.metabolites.h_i: 4})

In [9]:
lru.metabolites.Oxidized_CymA_Proteins_m.compartment = 'i'
lru.metabolites.Oxidized_CymA_Proteins_m.id = 'Oxidized_CymA_Proteins_i'
lru.metabolites.Reduced_CymA_Proteins_m.compartment = 'i'
lru.metabolites.Reduced_CymA_Proteins_m.id = 'Reduced_CymA_Proteins_i'

In [10]:
lru.remove_reactions(['ATPSYN_RXN[CCO_EXTRACELLULAR_CCO_CYTOSOL]_ATP/WATER/PROTON//ADP/Pi/PROTON.63.'])

In [11]:
lru.objective = lru.reactions.get_by_id('ATPSYN_RXN[CCO_MIT_IMEM]_ATP/WATER/PROTON//ADP/Pi/PROTON.46.')

In [15]:
# prevent unrealistic solutions that lead to a P O ratio of 1
lru.reactions.get_by_id('FORMYLTETRAHYDROFOLATE_DEHYDROGENASE_RXN').lower_bound = 0

In [12]:
lru.slim_optimize()

300.99999999985687

In [16]:
cobra.io.write_sbml_model(lru, os.path.join(species, "metabolic_model", "compartment_model_mod.sbml.gz"))